# Optimisation of LCA models

* Minimise environment impact and/or cost of a network of processes 
* OpenLCA for lifecycle assessment data
* Pyomo for mathematical optimisation
* Jupyter notebooks for prototyping
* Qt GUI for configuration and solution

# OpenLCA

* Java application built by Greendata
* Front-end to life-cycle database of processes and flows
* Ecoinvent system database has ~18000 processes ~7000 flows
* System processes broken down into elementary flows
* Contains methodologies to calculate impact of system
* Cannot feasibly optimise over multiple alternatives

# Pyomo

* An algebraic modelling langugage like GAMS, AIMMS
* Implemented as an object model in Python package `pyomo`
* Eases translation of mathematical optimisation problem to actual implementation
* Optimisation problems are formulated by building concrete or abstract model


# Mola

Python package builds optimisation models using openLCA data

| Module | Description |
| ------ | ----------- |
| `mola.dataimport` | Convert openLCA Ecoinvent system db to sqlite |
| `mola.dataview` | Build SQL to query sqlite db |
| `mola.specificationv5` | Classes each containing abstract Pyomo model |
| `mola.build` | Configure, populate, solve optimisation problem |

## Example

`mola.dataview` is a module of functions to build SQL and return Pandas DataFrame.

In [3]:
import mola.dataview as dv
dbconn = dv.get_sqlite_connection()

In [5]:
dv.get_processes(dbconn, name=['%oil%'])

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE '%oil%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,138081,956ebeef-370e-34bb-ac83-11a10bba21d1,"ethylvinylacetate production, foil | ethylviny...",LCI_RESULT,Rest-of-World,138083
1,179089,cd35ba47-c9bd-34b3-9b1b-83a6dfd88dfe,"coconut oil production, crude | coconut oil, c...",LCI_RESULT,Rest-of-World,179091
2,191391,71048d7d-a21a-3aec-8b28-7fe7831f806e,"treatment of waste cooking oil, purified, este...",LCI_RESULT,France,191393
3,503149,d3d48e94-3144-3beb-99ff-f1a50026e355,"heat and power co-generation, oil | electricit...",LCI_RESULT,Southwest Power Pool,503151
4,505198,35ef2809-6de5-3443-805a-3b8eb71d0e81,"heat and power co-generation, oil | heat, dist...",LCI_RESULT,Southwest Power Pool,505200
...,...,...,...,...,...,...
599,35861212,e5f11aa5-33e1-333b-8bac-11e8bf56a53a,"fatty acid production, from palm oil | fatty a...",LCI_RESULT,Europe,35861214
600,35877604,d27e06fd-e3c6-347d-aa19-6c28d21b2143,"electricity production, oil | electricity, hig...",LCI_RESULT,"China, Guizhou (贵州)",35877606
601,35980270,2414d79e-f597-4b2a-aad2-69527772b2b1,"fishmeal and fish oil production, 63-65% prote...",LCI_RESULT,Rest-of-World,35980272
602,35982319,1abebbc3-317a-4de0-8b83-8c278f31a871,"fishmeal and fish oil production, 63-65% prote...",LCI_RESULT,Rest-of-World,35982321


## Specification

Decorates a `pyomo` abstract model so that we can build an interface using introspection.

```
class Specification():
    user_defined_sets = {'A': {'doc': 'test set', 'lookup': False}}
    user_defined_parameters = {'Q': {'index': ['A'], 'doc': 'test parameters'}}

    def __init__():
        self.abstract_model = AbstractModel()
        # build user defined and db parameters
        
    def populate(json_files): # populates abstract model
        dp = DataPortal()
        
```

# MolaQT

Front-end to Mola using `pyqt`. Built around a `Controller` object for each model

```
class Controller(QWidget):

  def __init__(spec):
      self.spec = spec
      self.sets = spec.get_default_sets()
      self.parameters = spec.get_default_parameters()
      self.lookup = LookupTables(self.db) # if required
  
  def get_config(self): # get current state
```

## StandardController

Aims to Mirror Pyomo modelling language using a collection of Qt Widgets

```
class StandardController(Controller):

  def __init__(spec):
       self.sets_editor = SetsEditor(self.sets, self.lookup)
       self.parameters_editor = ParametersEditor(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
  
```

## CustomController

High-level widgets tied to a particular model and workflow. Hardcodes the model to the UI.

```
class StandardController(Controller):

  def __init__(spec):
       self.objectives = ObjectivesWidget(self.sets, self.lookup)
       self.ProcessFlow = ProcessFlow(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
 ```

## DashboardController

* Not yet implemented
* No need to use tabbed interface
* Could create a single dashboard if model is sufficiently simple
* Incorporate or export output

# Demo of molaqt

* General overview: Github, DB, General Specification, Models
* AIMMS tutorial problem: interface and model construction
* Orange production and transport: openLCA data (StandardController)
* Orange production and transport: openLCA data (CustomController)

# Development work

* Tasks: pyomo.network - MultiTaskController, ScheduleController, DashboardController
* Storage: AIMMS advanced example, Sargent papers, STNs, RTNs
* Output: current module to convert pyomo output to DataFrames for export, graphing
* More sophistical openLCA models e.g. oil to diesel conversion
* Dynamic model generation?